# Matplotlib and Seaborn

### Handy hints 

* Some of the plotting libraries we use need to communicate a lot of data to the browser. Depending on which version of Jupyter you are running, you may need to launch this notebook with a higher data rate limit: `jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000`

* In general, we are using plotting libraries that return objects encapsulating the plot. You can check the type of these returned objects with `type()`. Jupyter's tools for exploring objects and methods will also be useful: the `?` and `??` operators, and tab autocompletion.

## Setup 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This causes Jupyter to display any matplotlib plots directly in the notebook
# It also works for seaborn, since seaborn uses matplotlib to render plots
%matplotlib inline
from matplotlib import pyplot as plt

Be aware that Seaborn automatically changes Matplotlib's defaults *on import*. Not only your Seaborn plots, but also your Matplotlib plots, will look different once Seaborn is imported. If you don't want this behaviour, you can call `sns.reset_orig()` after import, or `import seaborn.apionly as sns` in the first place.

In [ ]:
import seaborn as sns

### Housing data 

In [ ]:
sales = pd.read_csv("housing-data-10000.csv", 
                    usecols=['id','date','price','zipcode','lat','long',
                             'waterfront','view','grade','sqft_living','sqft_lot'],
                    parse_dates=['date'], 
                    dtype={'zipcode': 'category',
                           'waterfront': 'bool'})

In [ ]:
sales.head()

In [ ]:
sales.dtypes

Note that as well as specifying that the `date` field should be parsed as a date, we specified that certain variables are categorical (as opposed to integers). Some plotting commands understand pandas DataFrames and will treat categorical variables differently to numerical variables.

### Toy data 

In [ ]:
from io import StringIO

data_string = """name	number	engine_type	colour	wheels	top_speed_mph	weight_tons
Thomas	1	Tank	Blue	6	40	52
Edward	2	Tender	Blue	14	70	41
Henry	3	Tender	Green	18	90	72.2
Gordon	4	Tender	Blue	18	100	91.35
James	5	Tender	Red	14	70	46
Percy	6	Tank	Green	4	40	22.85
Toby	7	Tank	Brown	6	20	27
Emily	12	Tender	Green	8	85	45
Rosie	37	Tank	Purple	6	65	37
Hiro	51	Tender	Black	20	55	76.8"""

trains = pd.read_table(StringIO(data_string))
trains['size'] = pd.cut(trains['weight_tons'], 3, labels=['Small','Medium','Big'])

trains

## Matplotlib 

Matplotlib is the oldest and still the fundamental plotting library in Python. It has a huge range of capabilities. Many other libraries (including Seaborn) use Matplotlib as a back-end renderer.

Today we're focussing on plotting tabular data. We won't touch on all Matplotlib's capabilities. If you want to see more of the range of things Matplotlib can do, you can look through the [Matplotlib gallery](https://matplotlib.org/gallery.html.), or try out this excellent [Matplotlib tutorial](https://www.labri.fr/perso/nrougier/teaching/matplotlib/).

An example Matplotlib plot with legend and annotation:

In [ ]:
x = [1,2,3,4,5]
y = [2,5,10,17,26]
y2 = [1,4,9,11,9]

fig, ax = plt.subplots()
ax.plot(x, y, c='blue', label='Projected')
ax.scatter(x, y2, c='red', label='Actual')
fig.legend()
ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

fig.savefig('example_matplotilb.png')

## Seaborn 

Seaborn builds on Matplotlib. Some nice features are:

- works directly with Pandas dataframes, concise syntax
- lots of plot types, including some more advanced options
- statistical plotting: many plots do summary statistics for you
- good default aesthetics and easy control of aesthetics
- using Matplotlib gives benefits of Matplotlib - many backends, lots of control
- underlying Matplotlib objects are easy to tweak directly

For completeness, here's the plot we made before:

In [ ]:
df = pd.DataFrame({
    'Time': [1,2,3,4,5],
    'Projected': [2,5,10,17,26],
    'Actual': [1,4,9,11,9]
})

fig, ax = plt.subplots()
sns.scatterplot(data=df, x='Time', y='Actual', color='red', ax=ax)
sns.lineplot(data=df, x='Time', y='Projected', color='blue', ax=ax)

ax.set_ylabel('Huge profits')

ax.annotate("where it all went wrong", 
                                 xy=(3,10), xytext=(1,12),
                                 arrowprops={'width':2})

Notice that we can add changes like annotations in exactly the same way, as we have Matplotlib Figure and Axes objects.

### Seaborn and Pandas 

In some cases we can use Seaborn by passing in lists (or arrays or series) directly:

In [ ]:
sns.barplot(x=['A','B','C'], y=[33,44,65])

However Seaborn is aware of Pandas and it is very common to use Seaborn directly with DataFrames. Plotting functions can take a DataFrame as their `data` parameter and then refer directly to column names:

In [ ]:
sns.barplot(data=trains, x='engine_type', y='top_speed_mph')

Here Seaborn has interpreted the `x` and `y` arguments as field names in the supplied DataFrame. Notice also that Seaborn has performed the summary statistics for us - in this case, using the default `estimator`, which is `mean()`. 

Notice also what happens if we simply swap the `x` and `y` parameters. Seaborn will automatically deduce that the categorical or string-like variable must be the bar labels, and the numeric variable must be the numeric axis:

**Exercise:** create a (vertical) bar plot using the `sales` data, showing how house prices vary with the value of the property `grade`.

Bar plots are often deplored as a way of showing statistical estimates, as only the top of the bar is really important, and the bar itself is a visual distraction. A point plot is an alternative, and plots like box plots can show more information. Several other plot types also show distributional information within categories.

**Exercise:** reproduce the plot you just made, using instead each of the Seaborn functions:

- pointplot()
- boxplot()
- violinplot()  (try the `scale` parameter)
- lvplot()
- stripplot() [SEE WARNING]  (try the `jitter` parameter)
- swarmplot() [SEE WARNING]

Note what sort of information about the distribution is shown by each.

WARNING: `stripplot()` and `swarmplot()` will plot individual data points. There are too many house sales to easily display in this way - you may want to subsample the dataframe with e.g.  `data=sales.sample(100)`.

Demo: Let's try making a horizontal bar plot of `price` against `grade` by putting `grade` on the y axis:

### Hue 

Many Seaborn plotting functions take a `hue` parameter. This colours the plot elements by some categorical variable, but more than this, summary statistics are calculated for each level of the hue variable.

In [ ]:
# It appears that my hypothesis that more wheels make you faster is flawed
sns.lmplot(data=trains, x='wheels', y='top_speed_mph', 
           hue='engine_type', palette=['red', 'blue'])

**Exercises:** 
* Create an `lmplot` of house price against living area. Do this without a `hue` parameter, then add in `waterfront` as the `hue` parameter. What information is the hue giving in this case?
* Try adding the `hue` parameter to one of your previous plots of some other type - for instance, a box plot.

If we'd wanted to colour scatter points by some continuous variable, `hue` can be made to work, but doesn't really make sense as it is intended for discrete values. In this specific case, we could pass our colour variable down to the underlying Matplotlib scatter call via the `scatter_kws` parameter. We'll look more at this later. Or, if we're not trying to fit a linear model, we could just use Seaborn aesthetics around a Matplot scatter plot:

In [ ]:
sample = sales.sample(200)

with sns.axes_style('white'):
    fig, ax = plt.subplots()
    ax.scatter(x=sales['long'], y=sales['lat'], c=sales['price'].apply(np.log), 
               alpha=0.5, cmap='Reds')

For discrete color palettes, as needed by the `hue` parameter, Seaborn has a `color_palette()` function to generate a useful range of palettes. You can find [a tutorial on choosing color palettes here](https://seaborn.pydata.org/tutorial/color_palettes.html). 

### Compound plots 

Seaborn has some plotting functions which create more complex figures made of multiple subplots. These include `cat()`, `factorplot()`, `jointplot()`, `lmplot()` and `clustermap()`. Let's see a few examples:

In [ ]:
# jointplot shows a scatter or density plot, with marginal distributions
sns.jointplot(data=sales, x='sqft_living', y='price') #, kind='reg')

In [ ]:
# pairplot shows pairwise relationships between variables
# Note that a variable like engine_type would be ignored as it is not numeric
sns.pairplot(data=trains[['wheels', 'top_speed_mph', 'weight_tons']])

In [ ]:
# factorplot conditions different subplots on different variable values
# we map variables to row and column of a grid of plots (as well as to hue)
# in this example, we just use columns, and so have only one row
sns.factorplot(data=trains, kind='bar',
               x='size', y='top_speed_mph', 
               col='engine_type')

**Exercise:** design a plot using `sns.catplot()`, to show the relationship between house price and: grade, view, and whether the property is waterfront. Available channels of information are:
- x and y coordinates
- hue
- row and column of subplot (`row` and `col`)

You can set the `kind` parameter to the kind of plot you want to make: point, bar, count, box, violin, or stripplot.

You do not have to use all of these channels - in fact your plot may be difficult to take in if you do.

### Colour and Palettes

Seaborn has good colour options. There are a few ways we could want to use access colours:

* Specify an individual colour for some plot element. Matplotlib named colours can be used, or rgb values specified. Also check out the `sns.xkcd_rgb` dictionary of 954 named colours from the XKCD colour survey - for instance, `sns.xkcd['fire engine red']` is a colour.
* Specify a colormap, for mapping a continuous value to colour. All Matplotlib colormaps can be used by name. You can see these under the `plt.cm` module. Seaborn's `light_palette()` and `dark_palette()` functions can also generate custom colourmaps easily.
* Specify a discrete colour palette (a list of colours), for mapping a discrete or categorical variable to colour. In Seaborn, there is a distinction between colour palettes and colormaps. In general, you can create a colour palette by explicitly listing some colours, or by selecting a series of colours along some colormap. There are several functions, including `color_palette()`, `light_palette()`, `dark_palette()`, `diverging_palette()` and `xkcd_palette()`, which can produce many discrete colour palettes of whatever size you need. 

In [ ]:
# An example discrete colour palette of 7 colours, based on the XKCD colour "denim blue"
# palplot is a function to visualise a palette
palette = sns.light_palette("denim blue", n_colors=7, input='xkcd')
sns.palplot(palette)

In [ ]:
# Equivalently (to illustrate that we can use an rgb value directly)
denim_blue = sns.xkcd_rgb["denim blue"]
print(denim_blue)
palette = sns.light_palette(denim_blue, n_colors=7)
print(palette)
sns.palplot(palette)

**Exercise:** Try out the Seaborn function `choose_diverging_palette()` in your notebook (it requires no arguments). You can assign the result to a variable.

Let's try a heatmap. Unlike most Seaborn functions, heatmaps take data in wide form. We'll need to pivot our long-form data to get a table of numbers, indexed by two variables. The heatmap function will then transform each number to a colour via a continuous colourmap.

**Exercise:** Use `sales.pivot_table()` to produce a table showing average house prices. The x-axis (column headers) should be the values of the `grade` variable, and the rows (index) should be the values of the `view` variable. If you're new to Pandas, check the example below that uses the `trains` toy dataset.

In [ ]:
# Here's an example using the toy dataset
# Avoid looking at the details first if you want to solve the above without hints!

speed_table = trains.pivot_table(index='engine_type', columns='size', values='top_speed_mph', aggfunc=np.mean)
speed_table

In [ ]:
# we'll use a dark-background style so we can easily 
# distinguish the (transparent) null values from the default colourmap
with sns.axes_style('dark'):
    # vmin=0 starts our colour scale from zero, which makes sense for speeds
    sns.heatmap(speed_table, vmin=0, annot=True)

**Exercises:** 
* If you haven't already, produce a heatmap with the `sales` data as described above. You will probably want to leave off the annotations unless your plot is very large. 
* Specify a different colourmap using the `cmap` parameter to `heatmap`. An ascending (not diverging) colourmap is appropriate for prices that are all positive, but in the Jupyter notebook, it might be nice to pick a colourmap that is reversed so that the whiter colours are closer to zero. Matplotlib colourmaps ending in "_r" are reversed.
* House prices have a skewed distribution, and so our heatmap doesn't really show detail for the lower end of the scale. Try to plot a heatmap where colour is based on the *log* of the price.  

### Seaborn and Matplotlib 

Seaborn plotting functions call Matplotlib plotting functions, and we can pass arguments down to these underlying functions.

For instance, `lmplot()` calls `scatter()` and `plot()` to draw points and lines. We can pass arguments down using `scatter_kws` and `line_kws`:

In [ ]:
sns.lmplot(data=sales, x='grade', y='price',
           scatter_kws={'alpha': 0.3},
           line_kws={'linestyle': 'dashed', 'color': 'red'})

**Exercise:** Which underlying plotting function is called by your `factorplot` above? You can try passing arguments to it with factorplot's `kwargs`.

Let's look at the object returned by a Seaborn plot function.

In [ ]:
g = sns.countplot(trains['engine_type'])

In [ ]:
type(g)

This is a Matplotlib `Axes` object. We can use all the usual `Axes` methods to tweak the plot. What's more, if we have an existing `Axes` object, we can ask Seaborn to draw the plot onto it by passing it in as the `ax` parameter:

In [ ]:
fig, myaxes = plt.subplots(figsize=(7,5), facecolor=sns.xkcd_rgb['pale pink'])

sns.countplot(trains['engine_type'], color='purple', ax=myaxes)

And the plot is still attached to `fig`:

In [ ]:
fig

The simpler Seaborn plotting functions return `Axes` objects, and can take an `Axes` as a parameter. More complex functions like `jointplot()` and `factorplot()` need to make multiple subplots. What do they return?

In [ ]:
g = sns.pairplot(data=trains[['wheels', 'top_speed_mph', 'weight_tons']])

In [ ]:
type(g)

In [ ]:
type(g.fig)

In [ ]:
g.axes

In [ ]:
g.fig

As we can see, these more complex functions produce entire Matplotlib `Figure` objects, which can contain multiple `Axes`. The `Figure`, however, comes wrapped in a Seaborn class, which provides some convenient functions to manipulate the figure properties.

In [ ]:
# g.set sets a property on all Axes in the Figure
# set x-axis to log scale:
g.set(xscale='log')
g.fig

The Seaborn classes commonly returned are:

- JointGrid : a central bivariate plot with two marginal univariate plots. Used by `jointplot()`. 
- PairGrid : a grid of subplots for plotting pairwise relationships. Has convenience functions for mapping plots onto diagonal and non-diagonal elements. Used by `pairplot()`.
- FacetGrid : a grid of subplots showing the same relationship, conditioned on some variable across different subplots. Designed to map fields of a Pandas DataFrame to different rows, columns, and hues. Used by `factorplot()` and `lmplot()`.

It's possible to instantiate these classes yourself for custom plots.